In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from itertools import combinations
from sklearn.preprocessing import StandardScaler

#### Data Analysis

In [3]:
from datetime import datetime
health_df = pd.read_csv('healthcare_dataset.csv')
health_df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import chi2_contingency
from itertools import product
import scipy.stats as ss

# Dropping unnecessary columns for prediction (e.g., Name, Doctor, Date of Admission, etc.)
health_df.drop_duplicates()
health_df['Discharge Date'] = pd.to_datetime(health_df['Discharge Date'])
health_df['Date of Admission'] = pd.to_datetime(health_df['Date of Admission'])
health_df['Inpatient Days']= (health_df['Discharge Date'] - health_df['Date of Admission']).dt.days
health_df["Per Day Amount"] = health_df["Billing Amount"]/health_df["Inpatient Days"]
features = health_df.drop(columns=['Name', 'Doctor', 'Date of Admission', 'Discharge Date'])

label_encoded_columns = ['Hospital', 'Gender', 'Insurance Provider']
label_encoder = LabelEncoder()

for col in label_encoded_columns:
    features[col] = label_encoder.fit_transform(features[col])


ohe_columns = ['Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results']
ohe = OneHotEncoder()

array_hot_encoded = ohe.fit_transform(features[ohe_columns])
encoded_df = pd.DataFrame(array_hot_encoded.toarray(), columns=ohe.get_feature_names_out(['Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results']))
df = features.join(encoded_df)

# Drop the original columns
df.drop(['Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results'], axis=1, inplace=True)
df = df[df['Billing Amount'] > 0]
df = df[df['Inpatient Days'] >= 0]
scaler = StandardScaler()
#df['Age'] = scaler.fit_transform(df[['Age']])
#df['Inpatient Days'] = scaler.fit_transform(df[['Inpatient Days']])

print(df.columns)

Index(['Age', 'Gender', 'Hospital', 'Insurance Provider', 'Billing Amount',
       'Room Number', 'Inpatient Days', 'Per Day Amount', 'Blood Type_A+',
       'Blood Type_A-', 'Blood Type_AB+', 'Blood Type_AB-', 'Blood Type_B+',
       'Blood Type_B-', 'Blood Type_O+', 'Blood Type_O-',
       'Medical Condition_Arthritis', 'Medical Condition_Asthma',
       'Medical Condition_Cancer', 'Medical Condition_Diabetes',
       'Medical Condition_Hypertension', 'Medical Condition_Obesity',
       'Admission Type_Elective', 'Admission Type_Emergency',
       'Admission Type_Urgent', 'Medication_Aspirin', 'Medication_Ibuprofen',
       'Medication_Lipitor', 'Medication_Paracetamol', 'Medication_Penicillin',
       'Test Results_Abnormal', 'Test Results_Inconclusive',
       'Test Results_Normal'],
      dtype='object')


In [46]:
#df = df[df['Billing Amount'] > 0]
#df = df[df['Inpatient Days'] > 0]
print(min(df['Inpatient Days']))
from sklearn.feature_selection import SelectKBest, f_regression
X = df.drop('Billing Amount', axis=1)
y = df['Billing Amount']
selector = SelectKBest(f_regression, k=5)
X_new = selector.fit_transform(X, y)
print(X_new)

1
[[9.42814065e+03 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00]
 [5.60722121e+03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e+00]
 [1.86367307e+03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e+00]
 ...
 [9.86455883e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [5.40851539e+03 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [1.48523488e+02 1.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]]


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=3)
model.fit(X, y)
print()

In [4]:
from sklearn.feature_selection import mutual_info_regression
indep = ['Age', 'Hospital', 'Medical Condition_Arthritis', 'Medical Condition_Asthma',
       'Medical Condition_Cancer', 'Medical Condition_Diabetes',
       'Medical Condition_Hypertension', 'Medical Condition_Obesity', 'Room Number',
       'Inpatient Days', 'Per Day Amount', 'Gender_Female', 'Gender_Male',
       'Blood Type_A+', 'Blood Type_A-', 'Blood Type_AB+', 'Blood Type_AB-',
       'Blood Type_B+', 'Blood Type_B-', 'Blood Type_O+', 'Blood Type_O-',
       'Admission Type_Elective', 'Admission Type_Emergency',
       'Admission Type_Urgent', 'Medication_Aspirin', 'Medication_Ibuprofen',
       'Medication_Lipitor', 'Medication_Paracetamol', 'Medication_Penicillin',
       'Test Results_Abnormal', 'Test Results_Inconclusive',
       'Test Results_Normal']

mutual_info_matrix = np.zeros((len(indep)))

# Calculate mutual information for each independent-dependent variable pair
for i, var1 in enumerate(indep):
    X = df[[var1]]
    y = df['Billing Amount']
    mutual_info = mutual_info_regression(X, y)
    mutual_info_matrix[i] = mutual_info[0]  # Store the mutual info scalar

# Convert the matrix to a DataFrame for easier viewing
mutual_info_df = pd.DataFrame(mutual_info_matrix, index=indep, columns=['Billing Amount'])

# Display the mutual information DataFrame
print(mutual_info_df)

                                Billing Amount
Age                                   0.010071
Hospital                              0.101619
Medical Condition_Arthritis           0.019433
Medical Condition_Asthma              0.020265
Medical Condition_Cancer              0.019169
Medical Condition_Diabetes            0.016990
Medical Condition_Hypertension        0.020043
Medical Condition_Obesity             0.019355
Room Number                           0.096651
Inpatient Days                        0.092819
Per Day Amount                        2.782328
Gender_Female                         0.033836
Gender_Male                           0.033833
Blood Type_A+                         0.014424
Blood Type_A-                         0.015963
Blood Type_AB+                        0.015481
Blood Type_AB-                        0.019596
Blood Type_B+                         0.018025
Blood Type_B-                         0.016790
Blood Type_O+                         0.016643
Blood Type_O-

In [5]:
top_pairs = mutual_info_df.unstack().sort_values(ascending=False).head(32)
print(top_pairs)

Billing Amount  Per Day Amount                    2.782328
                Hospital                          0.101619
                Room Number                       0.096651
                Inpatient Days                    0.092819
                Gender_Female                     0.033836
                Gender_Male                       0.033833
                Admission Type_Urgent             0.030986
                Test Results_Normal               0.029913
                Test Results_Inconclusive         0.029732
                Admission Type_Emergency          0.029694
                Test Results_Abnormal             0.028954
                Admission Type_Elective           0.027933
                Medication_Ibuprofen              0.026810
                Medication_Penicillin             0.025665
                Medication_Lipitor                0.023112
                Medication_Paracetamol            0.020308
                Medical Condition_Asthma          0.0202

In [47]:
y = df['Billing Amount']
a =df.drop(columns = ['Billing Amount'])
#a = a.drop(columns = ['Per Day Amount'])

In [7]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Create a linear regression model
model = LinearRegression()

rfe = RFE(model, n_features_to_select=20)
rfe.fit_transform(a, y)
print(a.columns[rfe.get_support()])

Index(['Gender_Female', 'Gender_Male', 'Blood Type_O-',
       'Medical Condition_Arthritis', 'Medical Condition_Asthma',
       'Medical Condition_Cancer', 'Medical Condition_Diabetes',
       'Medical Condition_Hypertension', 'Medical Condition_Obesity',
       'Admission Type_Elective', 'Admission Type_Emergency',
       'Admission Type_Urgent', 'Medication_Aspirin', 'Medication_Ibuprofen',
       'Medication_Lipitor', 'Medication_Paracetamol', 'Medication_Penicillin',
       'Test Results_Abnormal', 'Test Results_Inconclusive',
       'Test Results_Normal'],
      dtype='object')


In [8]:
from sklearn.metrics import r2_score, root_mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2)
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))



0.2957819218355291
0.5292497851429239


In [9]:
b = a[['Gender_Female', 'Gender_Male', 'Blood Type_O+',
       'Medical Condition_Arthritis', 'Medical Condition_Asthma',
       'Medical Condition_Cancer', 'Medical Condition_Diabetes',
       'Medical Condition_Hypertension', 'Medical Condition_Obesity',
       'Admission Type_Elective', 'Admission Type_Emergency',
       'Admission Type_Urgent', 'Medication_Aspirin', 'Medication_Ibuprofen',
       'Medication_Lipitor', 'Medication_Paracetamol', 'Medication_Penicillin',
       'Test Results_Abnormal', 'Test Results_Inconclusive',
       'Test Results_Normal']]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2)
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

-0.000869444225867877
0.44630176585730674


In [11]:
from sklearn.linear_model import Lasso

# Create a Lasso regression model
model = Lasso(alpha=0.1)
model.fit(a, y)

# Get feature importance scores
importance = np.abs(model.coef_)
importance_df = pd.DataFrame({'Feature': a.columns, 'Importance': importance})

# Sort the DataFrame by importance scores
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

                           Feature    Importance
3                   Inpatient Days  5.702222e+03
11                   Blood Type_B+  2.238512e+02
20       Medical Condition_Obesity  2.036238e+02
17        Medical Condition_Cancer  1.779304e+02
18      Medical Condition_Diabetes  1.535252e+02
28           Medication_Penicillin  1.453163e+02
10                  Blood Type_AB-  1.392938e+02
13                   Blood Type_O+  1.371820e+02
14                   Blood Type_O-  1.357686e+02
26              Medication_Lipitor  1.139301e+02
9                   Blood Type_AB+  1.094888e+02
23           Admission Type_Urgent  7.466389e+01
31             Test Results_Normal  6.519040e+01
25            Medication_Ibuprofen  5.982356e+01
7                    Blood Type_A+  5.332922e+01
5                    Gender_Female  5.199033e+01
30       Test Results_Inconclusive  4.094645e+01
15     Medical Condition_Arthritis  3.951052e+01
21         Admission Type_Elective  3.074025e+01
24              Medi

In [12]:
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2)
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))


0.3019429284899875
0.5329550538888199


In [54]:
b = a[['Medical Condition_Arthritis', 'Blood Type_O+', 'Hospital',
       'Age', 'Blood Type_B+',
       'Medical Condition_Diabetes', 'Inpatient Days']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2)
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))
Feature Medical Condition_Arthritis with importance: 1730.5128
Feature Blood Type_O+ with importance: 1714.5136
Feature Hospital with importance: 1683.9190
Feature Age with importance: 1682.5838
Feature Blood Type_B+ with importance: 1647.4970
Feature Medical Condition_Diabetes with importance: 1636.3089
Feature Inpatient Days with importance: 1620.2044
Feature Blood Type_O- with importance: 1578.7868
Feature Blood Type_AB- with importance: 1560.9029
Feature Blood Type_A- with importance: 1513.1654
Feature Medical Condition_Cancer with importance: 1510.8546
Feature Gender with importance: 1508.0452
Feature Per Day Amount with importance: 1487.7135
Feature Blood Type_B- with importance: 1487.0563
Feature Room Number with importance: 1476.8901

0.00013468090775425878
0.44947060131507555


In [55]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(b, df['Billing Amount'], test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

#activation{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
#solver{‘lbfgs’, ‘sgd’, ‘adam’}
clf = MLPRegressor(hidden_layer_sizes=(400,), max_iter=1000, alpha = 0.005, learning_rate_init= 0.002, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)

feature_importance = np.abs(clf.coefs_[0]).sum(axis=1)
sorted_indices = np.argsort(feature_importance)[::-1]
for idx in sorted_indices[:15]:
    print(f"Feature {a.columns[idx]} with importance: {feature_importance[idx]:.4f}")
ypred = clf.predict(X_test)
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

Feature Age with importance: 2278.2685
Feature Inpatient Days with importance: 2084.3400
Feature Room Number with importance: 2030.3486
Feature Gender with importance: 1930.2020
Feature Insurance Provider with importance: 1174.2023
Feature Per Day Amount with importance: 1132.7449
Feature Hospital with importance: 1027.8828
0.4423929430803608


In [53]:
b = a[['Per Day Amount', 'Inpatient Days', 'Test Results_Abnormal']]

X_train, X_test, y_train, y_test = train_test_split(b, df['Billing Amount'], test_size=0.2, random_state=42)

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

#activation{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
#solver{‘lbfgs’, ‘sgd’, ‘adam’}

clf = MLPRegressor(hidden_layer_sizes=(400,), max_iter=1000, alpha = 0.005, learning_rate_init= 0.002, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)

ypred = clf.predict(X_test)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

0.9982774203302753
0.9768738005146085


C:\Users\kykyk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [49]:
df['Billing Amount']

bins = [0, 10000, 20000, 30000, 40000, float('inf')]
labels = [10, 20, 30, 40, 50]

df['cat bill'] = pd.cut(df['Billing Amount'], bins=bins, labels=labels, right=True)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
df['l'] = kmeans.fit_predict(df[['Billing Amount']])

centroids = kmeans.cluster_centers_

# Find the min and max values within each cluster
cluster_edges = {}
for cluster_label in df['l'].unique():
    # Filter the data for the current cluster
    cluster_data = df[df['l'] == cluster_label]['Billing Amount']
    
    # Find the min and max values for each cluster (edges)
    min_val = cluster_data.min()
    max_val = cluster_data.max()
    
    cluster_edges[cluster_label] = (min_val, max_val)

# Print the cluster edges
print("Cluster edges (min, max) for each cluster:")
for cluster_label, (min_val, max_val) in cluster_edges.items():
    print(f"Cluster {cluster_label}: Min = {min_val}, Max = {max_val}")
print(df['cat bill'])

Cluster edges (min, max) for each cluster:
Cluster 0: Min = 10363.66893067308, Max = 19878.29987681973
Cluster 4: Min = 29703.870474486597, Max = 39836.3848779898
Cluster 2: Min = 19879.862382196294, Max = 29699.31359663662
Cluster 1: Min = 39837.74085278687, Max = 52764.276736469175
Cluster 3: Min = 9.238787497393332, Max = 10362.621045769216
0        20
1        40
2        30
3        40
4        20
         ..
55495    10
55496    40
55497    30
55498    40
55499    10
Name: cat bill, Length: 55392, dtype: category
Categories (5, int64): [10 < 20 < 30 < 40 < 50]


In [30]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
b = a[['Inpatient Days', 'Per Day Amount', 'Room Number']]
X_train, X_test, y_train, y_test = train_test_split(a, df['cat bill'], test_size=0.2, random_state=42)

#print(X_train)
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.fit_transform(X_test)

#activation{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
#solver{‘lbfgs’, ‘sgd’, ‘adam’}

clf = MLPClassifier(hidden_layer_sizes=(400,), max_iter=1000, alpha = 0.1, learning_rate_init= 0.005, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)

feature_importance = np.abs(clf.coefs_[0]).sum(axis=1)
sorted_indices = np.argsort(feature_importance)[::-1]
for idx in sorted_indices[:15]:
    print(f"Feature {a.columns[idx]} with importance: {feature_importance[idx]:.4f}")
ypred = clf.predict(X_test)
print(accuracy_score(y_test, ypred))
#print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

Feature Room Number with importance: 22.1384
Feature Per Day Amount with importance: 20.7829
Feature Inpatient Days with importance: 17.3488
Feature Hospital with importance: 11.8769
Feature Insurance Provider with importance: 4.5040
Feature Admission Type_Emergency with importance: 1.3025
Feature Test Results_Normal with importance: 1.2586
Feature Gender with importance: 1.2385
Feature Test Results_Inconclusive with importance: 1.1880
Feature Test Results_Abnormal with importance: 1.1781
Feature Admission Type_Urgent with importance: 1.1730
Feature Admission Type_Elective with importance: 1.0709
Feature Medication_Penicillin with importance: 0.8545
Feature Medication_Paracetamol with importance: 0.7411
Feature Medical Condition_Diabetes with importance: 0.6937
0.37783193429009837


In [55]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
b = a[['Inpatient Days', 'Per Day Amount', 'Test Results_Abnormal', 'Hospital']]
X_train, X_test, y_train, y_test = train_test_split(b, df['cat bill'], test_size=0.2, random_state=42)

#print(X_train)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

#activation{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
#solver{‘lbfgs’, ‘sgd’, ‘adam’}

clf = MLPClassifier(hidden_layer_sizes=(400,), max_iter=1000, alpha = 0.005, learning_rate_init= 0.002, activation='relu', solver='adam', random_state=12)
clf.fit(X_train, y_train)

ypred = clf.predict(X_test)
print(accuracy_score(y_test, ypred))

0.9805939164184493


In [34]:
from sklearn.inspection import permutation_importance

# Assuming clf is your trained MLPRegressor and X_test, y_test are your test data
result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=42)

# Get mean importance scores for each feature
importance_scores = result.importances_mean

# Pair feature names with their importance scores
feature_importances = list(zip(X_test.columns, importance_scores))

# Sort features by importance
feature_importances.sort(key=lambda x: x[1], reverse=True)

# Display the most important features
print("Feature importances:")
for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Feature importances:
Per Day Amount: 0.8418
Inpatient Days: 0.2847
Test Results_Abnormal: 0.0162
Admission Type_Urgent: 0.0157
Admission Type_Elective: 0.0153
Admission Type_Emergency: 0.0149
Test Results_Inconclusive: 0.0149
Test Results_Normal: 0.0145
Medication_Aspirin: 0.0099
Medication_Paracetamol: 0.0093
Medication_Penicillin: 0.0084
Medication_Ibuprofen: 0.0081
Medication_Lipitor: 0.0078
Medical Condition_Diabetes: 0.0077
Medical Condition_Obesity: 0.0073
Medical Condition_Hypertension: 0.0067
Medical Condition_Asthma: 0.0066
Medical Condition_Cancer: 0.0063
Blood Type_B-: 0.0058
Medical Condition_Arthritis: 0.0056
Blood Type_A+: 0.0054
Blood Type_AB-: 0.0047
Blood Type_A-: 0.0046
Blood Type_O-: 0.0043
Blood Type_B+: 0.0040
Blood Type_O+: 0.0039
Blood Type_AB+: 0.0032
Gender: 0.0008
Hospital: 0.0000
Age: -0.0000
Insurance Provider: -0.0002
Room Number: -0.0002
